# Gather

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
import json
import string
import re
import random
seed = 1234

# change display setting
pd.set_option('display.max_colwidth', None)

In [2]:
# get tweet image predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url, allow_redirects=True)
open('image_predictions.tsv', 'wb').write(r.content)

335079

In [3]:
# twitter authentication 
import tweepy

consumer_key = 'TT2D0E2aIX0VEMNDs1ZdRll7W'
consumer_secret = '7Qv623gWuw4H8gvIWtHJ4ZB8gw6kz6cRdllGP8bzYIAKxYTDHU'
access_token = '1322275579496927233-6yA5Xz158culvlLTkTJND36bdSIsJR'
access_secret = 'fHqvXqAhZVMGovtvXF6AoSK2tuoDul5bIrNCoYDADjrkJ'


In [242]:
# Check Twiter ID
df_tw = pd.read_csv('twitter-archive-enhanced-2.csv')

In [6]:
df_tw.head(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None


In [58]:
# Query the Twitter API and store the result(Json Data) in text file

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


tweet_ids = []
for id_ in df_tw.tweet_id: tweet_ids.append(str(id_))
fail_list = []

with open('tweet_json.txt', 'w') as file:
    for id_ in tweet_ids:
        try:
            tweet = api.get_status(id_, tweet_mode='extended')
            json.dump(tweet, file)
            file.write('\n')
            print('Success', id_)
        except tweepy.TweepError as e:
            print("Fail", id_)
            fail_list.append(id_)


Success 892420643555336193
Success 892177421306343426
Success 891815181378084864
Success 891689557279858688
Success 891327558926688256
Success 891087950875897856
Success 890971913173991426
Success 890729181411237888
Success 890609185150312448
Success 890240255349198849
Success 890006608113172480
Success 889880896479866881
Success 889665388333682689
Success 889638837579907072
Success 889531135344209921
Success 889278841981685760
Success 888917238123831296
Success 888804989199671297
Success 888554962724278272
Fail 888202515573088257
Success 888078434458587136
Success 887705289381826560
Success 887517139158093824
Success 887473957103951883
Success 887343217045368832
Success 887101392804085760
Success 886983233522544640
Success 886736880519319552
Success 886680336477933568
Success 886366144734445568
Success 886267009285017600
Success 886258384151887873
Success 886054160059072513
Success 885984800019947520
Success 885528943205470208
Success 885518971528720385
Success 885311592912609280
Succ

Rate limit reached. Sleeping for: 225


Success 759099523532779520
Success 759047813560868866
Success 758854675097526272
Success 758828659922702336
Success 758740312047005698
Success 758474966123810816
Success 758467244762497024
Success 758405701903519748
Success 758355060040593408
Success 758099635764359168
Success 758041019896193024
Success 757741869644341248
Success 757729163776290825
Success 757725642876129280
Success 757611664640446465
Success 757597904299253760
Success 757596066325864448
Success 757400162377592832
Success 757393109802180609
Success 757354760399941633
Success 756998049151549440
Success 756939218950160384
Success 756651752796094464
Success 756526248105566208
Success 756303284449767430
Success 756288534030475264
Success 756275833623502848
Success 755955933503782912
Success 755206590534418437
Success 755110668769038337
Success 754874841593970688
Success 754856583969079297
Success 754747087846248448
Success 754482103782404096
Success 754449512966619136
Success 754120377874386944
Fail 754011816964026368
Succ

Rate limit reached. Sleeping for: 685


Success 677314812125323265
Success 677301033169788928
Success 677269281705472000
Success 677228873407442944
Success 677187300187611136
Success 676975532580409345
Success 676957860086095872
Success 676949632774234114
Success 676948236477857792
Success 676946864479084545
Success 676942428000112642
Success 676936541936185344
Success 676916996760600576
Success 676897532954456065
Success 676864501615042560
Success 676821958043033607
Success 676819651066732545
Success 676811746707918848
Success 676776431406465024
Success 676617503762681856
Success 676613908052996102
Success 676606785097199616
Success 676603393314578432
Success 676593408224403456
Success 676590572941893632
Success 676588346097852417
Success 676582956622721024
Success 676575501977128964
Success 676533798876651520
Success 676496375194980353
Success 676470639084101634
Success 676440007570247681
Success 676430933382295552
Success 676263575653122048
Success 676237365392908289
Success 676219687039057920
Success 676215927814406144
S

In [125]:
len(fail_list), len(df_tw)

(25, 2356)

In [115]:
# Make a dataframe from the text file
df_list = []

with open ('tweet_json.txt', 'r') as json_file:
    for data in json_file:
        tweet_id = json.loads(data)['id']
        retweet_count = json.loads(data)['retweet_count']
        fave_count = json.loads(data)['favorite_count']

        df_list.append({'tweet_id': str(tweet_id),
                        'retweet_count' : int(retweet_count),
                        'favorite_count' : int(fave_count)})
df_tw_count = pd.DataFrame(df_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])

In [122]:
df_tw_count.head(2)

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7491,35483
1,892177421306343426,5559,30705


In [120]:
# save data
df_tw_count.to_csv('twitter_count.csv', index = False)

# Assess

### 1. twitter archive data

In [189]:
# Quick look
df_tw.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2155,669603084620980224,NaN,NaN,2015-11-25 19:46:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Very human-like. Cute overbite smile *finger to earpiece* I'm being told that the dog is actually on the right\n10/10 https://t.co/MSIbWu4YYs,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669603084620980224/photo/1,10,10,None,None,None,None,None
1568,687818504314159109,NaN,NaN,2016-01-15 02:08:05 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",With great pupper comes great responsibility. 12/10 https://t.co/hK6xB042EP,NaN,NaN,NaN,https://twitter.com/dog_rates/status/687818504314159109/photo/1,12,10,None,None,None,pupper,None
1539,689661964914655233,NaN,NaN,2016-01-20 04:13:20 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Luca. He's a Butternut Scooperfloof. Glorious tongue. 12/10 would pet really well https://t.co/VcxZQPNZaV,NaN,NaN,NaN,https://twitter.com/dog_rates/status/689661964914655233/photo/1,12,10,Luca,None,None,None,None
280,839549326359670784,NaN,NaN,2017-03-08 18:52:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Winston. He knows he's a little too big for the swing, but he doesn't care. Kindly requests a push. 12/10 would happily oblige https://t.co/GuxEXTdnMu",NaN,NaN,NaN,https://twitter.com/dog_rates/status/839549326359670784/photo/1,12,10,Winston,None,None,None,None
1920,674265582246694913,NaN,NaN,2015-12-08 16:33:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Henry. He's a shit dog. Short pointy ears. Leaves trail of pee. Not fluffy. Doesn't come when called. 2/10 https://t.co/Pu9RhfHDEQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674265582246694913/photo/1,2,10,Henry,None,None,None,None


**Text columns shows that it is also including the URL of the tweet.**

In [9]:
# check the information
df_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

- Missing data: some data relates to reply/retweet are missing, but these are nullable data.<br>
&nbsp;- These retweet/reply should be removed from this table as we want only original ratings. Instead, I will include the number of retweet/reply based on twitter_count.csv.
- Dtype: <br>
&nbsp;- Timestamp is not time series data.<br>
&nbsp;- Each dog stage is not boolean. <br>
&nbsp;- Five columns relates to ID are not string/object.

In [228]:
df_tw[(pd.notnull(df_tw['in_reply_to_status_id'])) | (pd.notnull(df_tw['retweeted_status_id']))].shape

(259, 17)

**259 tweets are not the original and 2097 tweets are the original**

In [205]:
# Check duplication
df_tw['tweet_id'].duplicated().sum()

0

In [11]:
# Check the statistical description
df_tw.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


- rating_numerator: Min. 0 should be checked just in case. Max.1776 is suspiciously large, considering 75 percentile is just 12.<br>
- rating_denominator: Min. is 0. Max.170 should be also checked as normal denominator is 10.

In [20]:
# Further assessmen in numetator (Min. == 0)
df_tw[df_tw.rating_numerator == 0].text

315           When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag
1016    PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX
Name: text, dtype: object

**Min. 0 is correct.**

In [22]:
# Further assessmen in numetator (Max. == 1776)
df_tw[df_tw.rating_numerator == 1776].text

979    This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh
Name: text, dtype: object

**Max.1776 is also correct**

In [27]:
# Further assessmen in denominator (Min. == 0)
df_tw[df_tw.rating_denominator == 0].text

313    @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
Name: text, dtype: object

In [28]:
df_tw[df_tw.rating_denominator == 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",NaN,NaN,NaN,NaN,960,0,None,None,None,None,None


**This is not an original tweet and also 0 is not correct. This will be solved when we remove retweet/reply from this table.**

In [29]:
# Further assessmen in denominator (Max. == 170)
df_tw[df_tw.rating_denominator == 170].text

1120    Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
Name: text, dtype: object

In [348]:
# Further assessmen in denominator (> 10)
df_tw[df_tw.rating_denominator > 10][['tweet_id', 'rating_numerator', 'rating_denominator','text']]

,tweet_id,rating_numerator,rating_denominator,text
342,832088576586297345,11,15,@docmisterio account started on 11/15/15
433,820690176645140481,84,70,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd
784,775096608509886464,9,11,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…"
902,758467244762497024,165,150,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
1068,740373189193256964,9,11,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ"
1120,731156023742988288,204,170,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
1165,722974582966214656,4,20,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a
1202,716439118184652801,50,50,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq
1228,713900603437621249,99,90,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1
1254,710658690886586372,80,80,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12


**In general, denomitor seems calculated as "10 * the number of dog" and it can be bigger than 10. However, I see some incorrect denomitor as well. Also where the denominator is wrong, the numerator tends to be wrong together.**<br>

From here, I will check each column further.

In [33]:
# check colomn name
df_tw.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [49]:
df_tw[df_tw.floofer == 'floofer'].text

46       Meet Grizzwald. He may be the floofiest floofer I ever did see. Lost eyes saving a schoolbus from a volcano erpuption. 13/10 heroic as h*ck https://t.co/rf661IFEYP
200     At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk
582                                              This is Doc. He takes time out of every day to worship our plant overlords. 12/10 quite the floofer https://t.co/azMneS6Ly5
774                                         Atlas rolled around in some chalk and now he's a magical rainbow floofer. 13/10 please never take a bath https://t.co/nzqTNw0744
984                              This is Blu. He's a wild bush Floofer. I wish anything made me as happy as bushes make Blu. 12/10 would frolic with https://t.co/HHUAnBb6QB
1022                                   Here's a golden floofer helping with the groceries. Bed got in way. Still 11/10 helpful af (vid 

**According to the Dogtionary, "floof" is synonym of "floofer".**

In [120]:
# check if floof is included in the floofer category
df_tw[(df_tw.text.str.contains('floof')) & (df_tw.floofer != 'floofer')][['tweet_id', 'text', 'floofer']]

,tweet_id,text,floofer
25,887101392804085760,This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp,None
53,881906580714921986,This is Rey. He's a Benebop Cumberfloof. 12/10 dangerously pettable https://t.co/503CgWbhxQ,None
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,None
622,796080075804475393,This is Yogi. He's 98% floof. Snuggable af. 12/10 https://t.co/opoXKxmfFm,None
630,794332329137291264,This is Nimbus (like the cloud). He just bought this fancy af duck raincoat. Only protects one ear tho. 12/10 so h*ckin floofy https://t.co/SIQbb8c3AU,None
740,780543529827336192,Here's a perturbed super floof. 12/10 would snug so damn well https://t.co/VG095mi09Q,None
790,773922284943896577,"This is Heinrich (pronounced ""Pat""). He's a Botswanian Vanderfloof. Snazzy af bandana. 12/10 downright puptacular https://t.co/G56ikYAqFg",None
959,751251247299190784,This is Gilbert. He's being chased by a battalion of miniature floof cows. 10/10 we all believe in you Gilbert https://t.co/wayKZkDRTG,None
1001,747933425676525569,Pls don't send more sherks. I don't care how seemingly floofy they are. It does me so much frighten. Thank u. 11/10 https://t.co/oQqlOsla4R,None
1161,723912936180330496,This is Sugar. She's a Bolivian Superfloof. Spherical af. 12/10 would never let go of https://t.co/AhMfUu6Onm,None


**Some tweet including "floof" is not coreectly reflected in the stage category "floofer". These should be updated.**

In [135]:
# Check dog name with very short name
df_tw[df_tw.name.str.len() <= 2].name.unique()

array(['a', 'Mo', 'Bo', 'an', 'O', 'Al', 'my', 'by', 'Ed', 'JD', 'Jo'],
      dtype=object)

**Some of them looks like not a name such as "a".**

In [136]:
# Check names　
df_tw.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such',
       'Maya', 'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey',
       'Lilly', 'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella',
       'Grizzwald', 'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey',
       'Gary', 'a', 'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey',
       'Duddles', 'Jack', 'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow',
       'Terrance', 'Aja', 'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict',
       'Venti', 'Goose', 'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian',
       'Walter', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Dawn', 'Boomer', 'Cody', 'Rumble', 'Clifford',
       'quite', 'Dewey', 'Scout', 'Gizmo', 'Cooper', 'Harold', 'Shikha',
       'Jamesy', 'Lili', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Nept

In [154]:
# I see some pronoun and articles in the name column. 
list_pronoun = ['my','his','her','this','the']

In [153]:
df_tw[df_tw.name.isin(list_pronoun)][['tweet_id', 'name', 'text']]

,tweet_id,name,text
852,765395769549590528,my,This is my dog. Her name is Zoey. She knows I've been rating other dogs. She's not happy. 13/10 no bias at all https://t.co/ep1NkYoiwB
992,748692773788876800,his,That is Quizno. This is his beach. He does not tolerate human shenanigans on his beach. 10/10 reclaim ur land doggo https://t.co/vdr7DaRSa7
1120,731156023742988288,this,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv
1527,690360449368465409,the,Stop sending in lobsters. This is the final warning. We only rate dogs. Thank you... 9/10 https://t.co/B9ZXXKJYNx
1603,685943807276412928,the,This is the newly formed pupper a capella group. They're just starting out but I see tons of potential. 8/10 for all https://t.co/wbAcvFoNtn
1797,677269281705472000,the,This is the happiest pupper I've ever seen. 10/10 would trade lives with https://t.co/ep8ATEJwRb
1815,676613908052996102,the,This is the saddest/sweetest/best picture I've been sent. 12/10 😢🐶 https://t.co/vQ2Lw1BLBF
2037,671561002136281088,the,This is the best thing I've ever seen so spread it like wildfire &amp; maybe we'll find the genius who created it. 13/10 https://t.co/q6RsuOVYwU
2212,668587383441514497,the,Never forget this vine. You will not stop watching for at least 15 minutes. This is the second coveted.. 13/10 https://t.co/roqIxCvEB3
2345,666063827256086533,the,This is the happiest dog you will ever see. Very committed owner. Nice couch. 10/10 https://t.co/RhUEAloehK


**Some name are registered wrongly. Also, these assessment shows that the incorrect names usually start with lower case.**

In [181]:
# Fileter names start with lowercase.
lower_list = list(string.ascii_lowercase)

df_tw[df_tw.name.str.startswith(tuple(lower_list))][['tweet_id', 'name', 'text']]

,tweet_id,name,text
22,887517139158093824,such,I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba
56,881536004380872706,a,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF
118,869988702071779329,quite,RT @dog_rates: We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10…
169,859196978902773760,quite,We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10 https://t.co/g2nSyGenG9
193,855459453768019968,quite,"Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective https://t.co/Xc7uj1C64x"
...,...,...,...
2349,666051853826850816,an,This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't play catch well. 2/10 https://t.co/v5A4vzSDdc
2350,666050758794694657,a,This is a truly beautiful English Wilson Staff retriever. Has a nice phone. Privileged. 10/10 would trade lives with https://t.co/fvIbQfHjIe
2352,666044226329800704,a,This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx
2353,666033412701032449,a,Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR


In [182]:
df_tw[df_tw.name.str.startswith(tuple(lower_list))][['tweet_id', 'name', 'text']][:10]

,tweet_id,name,text
22,887517139158093824,such,I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba
56,881536004380872706,a,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF
118,869988702071779329,quite,RT @dog_rates: We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10…
169,859196978902773760,quite,We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10 https://t.co/g2nSyGenG9
193,855459453768019968,quite,"Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective https://t.co/Xc7uj1C64x"
335,832645525019123713,not,There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10 \nhttps://t.co/dp5h9bCwU7
369,828650029636317184,one,"Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK"
542,806219024703037440,incredibly,We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10 https://t.co/IZidSrBvhi
649,792913359805018113,a,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq
682,788552643979468800,mad,RT @dog_rates: Say hello to mad pupper. You know what you did. 13/10 would pet until no longer furustrated https://t.co/u1ulQ5heLX


**The names are incorrectly registered.**

### 2. Image prediction data

In [183]:
df_image = pd.read_csv('image_predictions.tsv', delimiter = '\t')

In [190]:
df_image.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
561,677895101218201600,https://pbs.twimg.com/media/CWhd_7WWsAAaqWG.jpg,1,dalmatian,0.550702,True,kuvasz,0.060226,True,Great_Pyrenees,0.058631,True
1429,772877495989305348,https://pbs.twimg.com/ext_tw_video_thumb/772874595468795904/pu/img/t8gbjy2rA19xtQYR.jpg,1,tabby,0.218303,False,Norwegian_elkhound,0.138523,True,wombat,0.074217,False
1357,760641137271070720,https://pbs.twimg.com/media/Co5XExUWgAAL5L_.jpg,1,axolotl,0.132695,False,killer_whale,0.131113,False,sea_lion,0.069652,False
952,705066031337840642,https://pbs.twimg.com/media/CcjlzRkW0AMqmWg.jpg,1,Airedale,0.868658,True,Irish_terrier,0.027587,True,otterhound,0.025324,True
1032,711652651650457602,https://pbs.twimg.com/media/CeBMT6-WIAA7Qqf.jpg,1,llama,0.856789,False,Arabian_camel,0.098727,False,neck_brace,0.016377,False


**If the prediction is not a breed of dog, I will treat it as Nan regardless of the confidence interval.**

In [192]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


**Some data are missing.**

In [206]:
# Check duplication
df_image['tweet_id'].duplicated().sum()

0

**Data type of tweet_id should be object.**

In [193]:
df_image.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [199]:
df_tw_count = pd.read_csv('twitter_count.csv')

In [200]:
df_tw_count.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7491,35483
1,892177421306343426,5559,30705
2,891815181378084864,3681,23088
3,891689557279858688,7669,38774
4,891327558926688256,8273,37039


In [201]:
df_tw_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2331 non-null   int64
 1   retweet_count   2331 non-null   int64
 2   favorite_count  2331 non-null   int64
dtypes: int64(3)
memory usage: 54.8 KB


In [207]:
# Check duplication
df_tw_count['tweet_id'].duplicated().sum()

0

#### In conclusion, these are the issues.

#### Quality issues 
##### ◆Twitter archive data
&nbsp;-Wrong Data type: <br>
>&nbsp;Timestamp (->time series data)<br>
&nbsp;Dog stage (-> Boolean)<br>
&nbsp;Tweet_id (-> string/object)<br>

&nbsp;-Some denominators are incorrect.<br>
&nbsp;-Some numerators are incorrect.<br>
&nbsp;-"Floof" is not included in floofer stage.<br>
&nbsp;-Some dog names are incorrect.<br> 
##### ◆Image prediction
&nbsp;-Some predisctions are not a dog.<br>
&nbsp;-Tweet ID is not string/object.<br>
&nbsp;-Some data are missing.<br>
<br>
#### Tidiness issues
##### ◆Twitter archive data
&nbsp;-Text column includes URL of the tweets.<br>
&nbsp;-Retweets / replies are included.<br>
&nbsp;-The number of retweet / reply is in a separate dataframe.<br>

# Clean

In [570]:
# Copy data
df_clean = df_tw.copy()

#### ◆Tidiness issue : Twitter archive data - Retweets / replied are included.

To clean this efficiently, first I will remove non-original tweets(retweet or reply).
Define :remove non-original tweets and remove columns ().

In [571]:
# Code : remove non-original tweets
df_clean = df_clean[(pd.isnull(df_clean['in_reply_to_status_id'])) & (pd.isnull(df_clean['retweeted_status_id']))]

In [572]:
# Test
df_clean[(pd.notnull(df_clean['in_reply_to_status_id'])) | (pd.notnull(df_clean['in_reply_to_user_id']))|(pd.notnull(df_clean['retweeted_status_id']))|(pd.notnull(df_clean['retweeted_status_user_id']))|(pd.notnull(df_clean['retweeted_status_timestamp']))]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [573]:
# Code : drop irrelevant columns
df_clean = df_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis = 1).reset_index(drop = True)

In [574]:
# Test
df_clean.head(1)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None


#### ◆Quality issue : Twitter archive data - Wrong Data type

Define：Change data type<br>
1. Time stamp should be time series data. As I will remove retweeted_status_timestamp later, I will keep this as it is for now.<br> 
2. Each dog stage should be boolean.<br>
3. Tweet ID should be string/object. Other ID related columns will be removed later so that I will keep it as it is for now.


In [575]:
# Code : 1
df_clean.timestamp = pd.to_datetime(df_tw['timestamp']) 

In [576]:
# test : 1
df_clean.timestamp

0      2017-08-01 16:23:56+00:00
1      2017-08-01 00:17:27+00:00
2      2017-07-31 00:18:03+00:00
3      2017-07-30 15:58:51+00:00
4      2017-07-29 16:00:24+00:00
                  ...           
2092   2015-11-29 01:52:48+00:00
2093   2015-11-29 01:45:22+00:00
2094   2015-11-29 01:35:26+00:00
2095   2015-11-29 00:39:59+00:00
2096   2015-11-29 00:06:39+00:00
Name: timestamp, Length: 2097, dtype: datetime64[ns, UTC]

In [577]:
# Code : 2
dog_stage = ['doggo','floofer', 'pupper', 'puppo']

for stage in dog_stage:
    df_clean[stage] = df_clean[stage].str.contains(stage)

In [578]:
# test : 2 
for stage in dog_stage:
    print((df_clean[stage] == (df_tw[(pd.isnull(df_tw['in_reply_to_status_id'])) & (pd.isnull(df_tw['retweeted_status_id']))].reset_index(drop=True)[stage] == stage)).unique())

[ True]
[ True]
[ True]
[ True]


In [579]:
#Code : 3
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

In [580]:
# test : 1-3
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097 entries, 0 to 2096
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2097 non-null   object             
 1   timestamp           2097 non-null   datetime64[ns, UTC]
 2   source              2097 non-null   object             
 3   text                2097 non-null   object             
 4   expanded_urls       2094 non-null   object             
 5   rating_numerator    2097 non-null   int64              
 6   rating_denominator  2097 non-null   int64              
 7   name                2097 non-null   object             
 8   doggo               2097 non-null   bool               
 9   floofer             2097 non-null   bool               
 10  pupper              2097 non-null   bool               
 11  puppo               2097 non-null   bool               
dtypes: bool(4), datetime64[ns, UTC](1)

#### ◆Twitter archive data: Incorrect denominators

Define: Assess and correct visually where denominators are bigger than 10. As there is higher chance that when denominator is wrong, the numerator is also wrong, I will check and correct both of them.

In [581]:
# Code
df_clean[df_clean.rating_denominator > 10][['tweet_id','text','rating_numerator', 'rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
339,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
700,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
853,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
904,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
948,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
985,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1011,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99,90
1036,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80,80
1056,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45,50
1129,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60,50


In [582]:
correct_rating = {'tweet_id':['740373189193256964', '722974582966214656', '716439118184652801', '682962037429899265'],
                 'rating_numerator':[14, 13, 11, 10],
                 'rating_denominator':[10, 10, 10, 10]}

In [583]:
df_correct_rating = pd.DataFrame(correct_rating, columns = ['tweet_id', 'rating_numerator', 'rating_denominator'])

In [568]:
correct_rating['tweet_id']

['740373189193256964',
 '722974582966214656',
 '716439118184652801',
 '682962037429899265']

In [536]:
df_correct_rating

,tweet_id,rating_numerator,rating_denominator
0,740373189193256964,14,10
1,722974582966214656,13,10
2,716439118184652801,11,10
3,682962037429899265,10,10


In [569]:
for ID in df_correct_rating.twe:
    df_clean[df_clean.tweet_id == ID][['rating_numerator', 'rating_denominator']] = df_correct_rating[['rating_numerator', 'rating_denominator']]

C:\Users\Kaori\anaconda3\lib\site-packages\pandas\core\frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [552]:
df_tw[df_tw.tweet_id == 740373189193256964]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,None,None,None,None


In [546]:
df_clean2[df_clean.rating_denominator > 10][['tweet_id','text','rating_numerator', 'rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
339,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0,70.0
700,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0,150.0
853,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9.0,11.0
904,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0,170.0
948,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4.0,20.0
985,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50.0,50.0
1011,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0,90.0
1036,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0,80.0
1056,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0,50.0
1129,704054845121142784,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0,50.0


# Storing

In [ ]:
# save as twitter_archive_master.csv

# Analyzing and Visualizing

In [ ]:
# At least three (3) insights and one (1) visualization